# DICOM-RT Preprocessing (CT + RTSTRUCT masks + optional T1/T2 MRI)

- **CT** as the reference grid (origin, spacing, direction preserved)
- **ROI masks** rasterized from **RTSTRUCT** onto the CT grid
- Optional **T1** and/or **T2** MRI, resampled to the CT grid
- **Output** arrays saved as NumPy, with image channels stacked as **[CT, T1, T2]**  
  (any missing MR channel is a **zero-filled** array with the same spatial shape)

**Pipeline outline**
1) Discover DICOM series per patient (CT, RTSTRUCT, optional MR T1/T2)  
2) Load CT; keep geometry (origin, spacing, direction)  
3) Parse RTSTRUCT → ROI masks aligned to CT  
4) Load MRI (T1/T2 if available) and resample to CT grid   
5) Stack channels → `[CT, T1, T2]` with zeros for any missing MR  
6) Save NumPy arrays (+ optional metadata)  

In [ ]:
import numpy as np
import pandas as pd
import json
import rosamllib
import shutil
import SimpleITK as sitk
from pathlib import Path
from scipy.ndimage import label
from rosamllib.readers import DICOMLoader
from rosamllib.readers.dicom_nodes import InstanceNode, SeriesNode
from rosamllib.viewers import interactive_image_viewer
from tqdm.notebook import tqdm

In [ ]:
data_path = Path(r"\\rbrofs2\temp$\jhink\GTV_HN_Project\Data_Backup")

In [ ]:
tags_to_index=[
    "StudyDate", 
    "StudyDescription", 
    "SeriesDate", 
    "SeriesDescription", 
    "BodyPartExamined", 
    "Manufacturer", 
    "ManufacturerModelName", 
    "InstitutionName", 
    "PhysiciansOfRecord", 
    "ReferringPhysicianName", 
    "OperatorsName", 
    "ImageOrientationPatient", 
    "ImagePositionPatient", 
    "RTPlanLabel", 
    "RTPlanDate", 
    "PlanIntent", 
    "TreatmentMachineName", 
    "ApprovalStatus", 
    "ReviewerName", 
    ]

In [ ]:
# Load all DICOMs first
loader = DICOMLoader(data_path)
loader.load(tags_to_index)

In [ ]:
loader.get_summary()

In [ ]:
loader.get_modality_distribution()

In [ ]:
rtstructs = loader.query("INSTANCE", Modality="RTSTRUCT", PatientName="*", PatientID="*")
rtstructs.head()

In [ ]:
# Get all ROI names used in all the RTSTRUCT files (skip if they don't reference planning CT)
roi_names = {}
for indx, row in rtstructs.iterrows():
    sop_uid = row["SOPInstanceUID"]
    inst = loader.get_instance(sop_uid)
    referenced_cts = loader.get_referenced_nodes(inst, modality="CT", level="SERIES")
    if referenced_cts:
        rtstruct = loader.read_instance(sop_uid)
        roi_names[sop_uid] = rtstruct.get_roi_names()

In [ ]:
# Join all ROI names in one set (get only unique names)
roi_unique = set()
for value in roi_names.values():
    roi_unique = roi_unique | set(value)

# Lower all ROI names
lowered = [item.lower() for item in roi_unique]

In [ ]:
lowered

### ROI name normalization (alias mapping)

Planners often use slightly different names for the **same anatomical structure** (e.g., “gtv”, “g_gtv”, “gtv pre-op”).  
To make downstream code robust, we define a **canonical ROI name** (e.g., `"GTV"`) and map it to a list of **known aliases**.  
During preprocessing, we lowercase incoming RTSTRUCT names and match them against this alias list to pick the canonical label.

In [18]:
# Create a dictionary to store ROI names with corresponding aliases used
roi_classes = {"GTV":[
    'g_gtv_pre-op',
    'g_gtv pre-op',
    'g_gtv_preop',
    'g_gtvpreop',
    'g_gtv pre-op ln',
    'g_gtv_primary',
    'g_gtv_pre-op',
    'gtv_node_pre-op',
    'g_gtv',
    'g_gtv pre-op pet',
    'g_pre-op_gtv_df',
    'gtv final',
    'pre_gtv',
    'gtv pre-op',
    'g_gtvn_preop',
    'g_gtv pre-op',
    'gtv_preop',
    'g_gtvp_pre-op',
    'gtv 66gy',
    'preop_gtv1',
    'g_gtvn pre-op',
    'g_gtvn',
    'gtv',
    'gtv_preop_node',
    'g_gtvp',
    'g_gtv_mri+pet',
    'g_pre-op_gtv',
    'g_gtvp pre-op',
    'g_gtv_submntl'
]
}

In [19]:
def resamp(image: sitk.Image, new_spacing=(1.0, 1.0, 1.0), interpolator=sitk.sitkLinear):
    """
    Resample a SimpleITK image to isotropic spacing.
    
    Parameters
    ----------
    image : sitk.Image
        Input SimpleITK image.
    new_spacing : tuple of float
        Desired voxel spacing (isotropic if all elements equal).
    interpolator : sitk.InterpolatorEnum
        Interpolation method (sitk.sitkLinear for CT, sitk.sitkNearestNeighbor for masks).
    
    Returns
    -------
    sitk.Image
        Resampled image.
    """
    # Original spacing and size
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()

    # Compute new size (preserve physical size of image)
    new_size = [
        int(round(osz * ospc / nspc))
        for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)
    ]

    # Define resampler
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())  # fill background if needed

    return resample.Execute(image)

def resamp_to_ref(referenced_image, main_image):
    """
    Resample an image onto a reference image's grid (copying its geometry).

    This function takes `main_image` and resamples it so that its voxel grid
    exactly matches `referenced_image` in **origin**, **spacing**, **direction**,
    and **size**. Interpolation is set to `sitk.sitkLinear` (good for CT/MR
    intensities). Voxels that map outside the original `main_image` are filled
    with `0` via `SetDefaultPixelValue(0)`.

    Parameters
    ----------
    referenced_image : sitk.Image
        The image providing the target geometry. Its origin, spacing, direction,
        and size will be copied to the resampled output.
    main_image : sitk.Image
        The image to be resampled onto the reference grid.

    Returns
    -------
    sitk.Image
        `main_image` resampled so that it aligns voxel-for-voxel with
        `referenced_image`.
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(referenced_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetOutputSpacing(referenced_image.GetSpacing())
    resampler.SetOutputOrigin(referenced_image.GetOrigin())
    resampler.SetOutputDirection(referenced_image.GetDirection())
    resampler.SetSize(referenced_image.GetSize())
    resampler.SetDefaultPixelValue(
        0
    )  # Set default value for regions outside the original dose grid

    resampled_image = resampler.Execute(main_image)
    return resampled_image

In [ ]:
save_path = Path(r"\\rbrofs2\temp$\jhink\GTV_HN_Project\Data_Arrays_2mm")
save_path.mkdir(exist_ok=True)
(save_path/"images").mkdir(exist_ok=True)
(save_path/"masks").mkdir(exist_ok=True)
pid_mapper = {}
skip_list = []
indx = 0
# We iterate once per patient and materialize:
#   1) CT (resampled to 2.0mm)  → channel 0
#   2) T1 MRI (if present, resampled to CT grid) → channel 1, else zeros
#   3) T2 MRI (if present, resampled to CT grid) → channel 2, else zeros
#   4) One-hot masks from RTSTRUCT (resampled to 2.0mm, NN interp), with a background channel added
for patient in tqdm(loader, total=len(loader.dataset), desc="Processing patients..."):
    pid_mapper[indx] = {}

    # RTSTRUCT: get the structure set instance tied to this patient -----
    inst = loader.get_nodes_for_patient(patient, "INSTANCE", "RTSTRUCT")[0]
    rtstruct = loader.read_instance(inst.SOPInstanceUID)

    # CT: find the SERIES that RTSTRUCT references and read it -----
    # We assume only the planning CT is referenced by the RTSTRUCT.
    referenced_cts = loader.get_referenced_nodes(inst, modality="CT", level="SERIES")[0]

    # MR: enumerate all MR SERIES for this patient (we'll detect T1/T2 by description) -----
    for_referenced_mrs = loader.get_nodes_for_patient(patient, "SERIES", "MR")

    # IMAGE CHANNEL STACK
    image_arrays = []

    # Read CT series, resample to isotropic 2.0 mm, convert to NumPy
    ct = loader.read_series(referenced_cts.SeriesInstanceUID)[0]
    ct_resamp = resamp(ct, new_spacing=(2.0, 2.0, 2.0))
    ct_array = np.expand_dims(sitk.GetArrayFromImage(ct_resamp), axis=0)  # [C, D, H, W]
    image_arrays.append(ct_array)

    # Flags to track presence of T1/T2; if absent, we will zero-fill the channel
    t1_flag = False
    t2_flag = False

    # MR HANDLING
    # Iterate all MR series; assign to T1/T2 based on SeriesDescription substring.
    # Each MR is resampled onto the *resampled CT* grid (ct_resamp) for voxel-level alignment.
    for mrs in for_referenced_mrs:
        mr_series = loader.read_series(mrs.SeriesInstanceUID)[0]

        # T1 detection by SeriesDescription (lowercased).
        if "t1" in str(mrs.SeriesDescription).lower():
            t1_flag = True
            mr_series = resamp_to_ref(ct_resamp, mr_series)
            t1_array = np.expand_dims(sitk.GetArrayFromImage(mr_series), axis=0)   # [C, D, H, W]

        # T2 detection by SeriesDescription (lowercased)    
        if "t2" in str(mrs.SeriesDescription).lower():
            t2_flag = True
            mr_series = resamp_to_ref(ct_resamp, mr_series)
            t2_array = np.expand_dims(sitk.GetArrayFromImage(mr_series), axis=0) # [C, D, H, W]
    
    # If T1 missing, append a zero channel with same shape as CT; else append T1 array
    if t1_flag:
        image_arrays.append(t1_array)
    else:
        t1_array = np.zeros_like(ct_array)
        image_arrays.append(t1_array)
    
    # If T2 missing, append a zero channel with same shape as CT; else append T2 array
    if t2_flag:
        image_arrays.append(t2_array)
    else:
        t2_array = np.zeros_like(ct_array)
        image_arrays.append(t2_array)

    # If both MR channels are missing entirely, mark this patient as skipped and move on  
    if not t1_flag and not t2_flag:
        skip_list.append(rtstruct.PatientID)
        continue

    # Concatenate along channel axis to form [C=3, D, H, W] in the fixed order [CT, T1, T2]
    image_stack = np.concatenate(image_arrays, axis=0)  # [C, D, H, W]

    # Pull structure names as found in the RTSTRUCT. Build a mapping with lowercase keys
    # to support case-insensitive matching against `roi_classes` below.
    structure_names = rtstruct.get_structure_names()
    structure_names = {structure_name.lower():structure_name for structure_name in structure_names}
    class_structures = []
    class_structure_masks = []

    # Ensure RTSTRUCT uses the current CT as its referenced image (for correct rasterization)
    rtstruct.set_referenced_image(ct)

    # For each class label (key) in roi_classes, find the matching structure name (case-insensitive)
    for key, value in roi_classes.items():
        # get the structure name that matches the class label `key`
        matching_name = list(set(value).intersection(set(structure_names.keys())))[0]
        class_structures.append(structure_names[matching_name])
    
    # Rasterize each selected structure into a binary mask, resample to 2.0mm (NN), ensure uint8
    for structure in class_structures:
        mask = np.expand_dims(
            sitk.GetArrayFromImage(
                resamp(
                    rtstruct.get_structure_mask(structure), 
                    new_spacing=(2.0, 2.0, 2.0), 
                    interpolator=sitk.sitkNearestNeighbor
                ), 
                axis=-1
            ) > 0
        mask = mask.astype(np.uint8)
        class_structure_masks.append(np.expand_dims(mask, axis=0))
    mask_stack = np.concatenate(class_structure_masks, axis=0, dtype=np.uint8)

    # Add background channel
    background = 1 - np.sum(mask_stack, axis=0, keepdims=True)
    background = np.clip(background, 0, 1)
    mask_stack = np.concatenate([background, mask_stack], axis=0)            # [C+1, S, H, W]

    # PER-CASE METADATA INTO pid_mapper
    pid_mapper[indx]["PatientID"] = rtstruct.PatientID
    pid_mapper[indx]["Structures"] = class_structures
    pid_mapper[indx]["ImageDepth"] = ct_resamp.GetDepth()
    pid_mapper[indx]["ImageSize"] = ct_resamp.GetSize()
    pid_mapper[indx]["SOPInstanceUID"] = rtstruct.SOPInstanceUID
    pid_mapper[indx]["Images"] = save_path / "images" / f"img_{indx}.npy"
    pid_mapper[indx]["Masks"] = save_path / "masks" / f"mask_{indx}.npy"
    
    # SAVE NUMPY ARRAYS
    np.save(save_path / "images" / f"img_{indx}.npy", image_stack)
    np.save(save_path / "masks" / f"mask_{indx}.npy", mask_stack)
    indx += 1

In [ ]:
with open(save_path / "pid_mapper_.json", 'w') as file:
    json.dump(pid_mapper, file, indent=4, default=str)

In [ ]:
import pandas as pd
df = pd.DataFrame({
    "Index":list(pid_mapper.keys()), 
    "PatientID":[el["PatientID"] for el in list(pid_mapper.values())], 
    "Structures":[el["Structures"] for el in list(pid_mapper.values())],
    "ImageDepth":[el["ImageDepth"] for el in list(pid_mapper.values())],
    "ImageSize":[el["ImageSize"] for el in list(pid_mapper.values())],
    "RTSTRUCT_SOPInstanceUID":[el["SOPInstanceUID"] for el in list(pid_mapper.values())]
    })

In [ ]:
df.to_csv(save_path / "pid_mapper.csv")